In [ ]:
!pip install quantstats

**Market Value Calculation**

-We will calculate the market_value

In [ ]:
def cal_market_value(pos_df):
    """Calculate market value by multiplying quantity, mtm_price, and base_factor."""
    pos_df['market_value'] = pos_df['quantity'] * pos_df['mtm_price'] * pos_df['base_factor']
    return pos_df


**Getting the data for processing**

*   Read the file
*   Process the data if the columns exist



In [ ]:
def get_data(filepath):
    """Load the data and check for required columns."""
    try:
        pos_df = pd.read_csv(filepath)
        logging.info("File loaded successfully.")

        required_columns = ['quantity', 'mtm_price', 'base_factor', 'report_date']

        # Verify required columns
        if all(col in pos_df.columns for col in required_columns):
            pos_df = cal_market_value(pos_df)
            return pos_df
        else:
            missing_cols = [col for col in required_columns if col not in pos_df.columns]
            logging.error(f"Missing columns: {missing_cols}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {filepath}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

**Merging Position and Security Sheet**
*   Get postion data and security data
*   And merge pos_data and security_df using 'security_id' from pos_data and 'symbol' from security_df


In [ ]:
def merge_pos_security_sheet(filepath1, filepath2):
    # Load and prepare position data
    pos_data = get_data(filepath1)
    # pos_data=pd.read_csv(filepath1)
    # Load security sheet data
    security_df = pd.read_csv(filepath2)


    # Merge dataframes on 'security_id' from pos_data and 'symbol' from security_df
    merged_df = pd.merge(pos_data, security_df, left_on='security_id', right_on='symbol', how='inner')
    #print(merged_df['client_id'].unique())

    # Print merged dataframe for verification
    return merged_df




**Returns Calculation**
*   Filters out rows where filter_column
*   Find sum of market_value for the selected filter_column
*   Calculate the daily returs

In [ ]:
def cal_returns(df, filter_column):
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting to process data.")

    try:
        # Convert 'report_date' to datetime format to ensure datetime-like accessors work
        df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')

        # Filter the data based on the selected filter_column (e.g., 'asset_class_x')
        df = df[(df['asset_class_x'] != 'Loan') & (df['asset_class_y'] != 'Loan')]
        daily_returns = df.groupby(['report_date', filter_column])['market_value'].sum().reset_index()

        # Rename the 'market_value' column to 'total_market_value'
        daily_returns.rename(columns={'market_value': 'total_market_value'}, inplace=True)

        # Calculate daily returns as the percentage change in 'total_market_value'
        daily_returns['daily_returns'] = daily_returns.groupby(filter_column)['total_market_value'].pct_change()

        logging.info("Calculated daily returns.")

        # Format 'report_date' for JSON output only
        daily_returns_json = daily_returns.copy()
        daily_returns_json['report_date'] = daily_returns_json['report_date'].dt.strftime('%Y-%m-%d')

        # Convert to JSON format for output
        json_output = daily_returns_json[['report_date', filter_column, 'total_market_value', 'daily_returns']].to_json(orient='records', double_precision=2)
        logging.info("Converted data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        # Return original daily_returns for downstream functions and formatted JSON output
        return json_output, total_time, daily_returns

    except Exception as e:
        logging.error("Error processing data: %s", e)
        raise


In [ ]:
import pandas as pd
import time
import logging

def cal_monthly_returns(df, filter_column):
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting monthly returns calculation.")

    try:
        # Ensure 'report_date' is in datetime format
        df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')

        # Drop rows with invalid dates or missing values
        df.dropna(subset=['report_date', 'quantity', 'mtm_price', 'base_factor'], inplace=True)

        # Filter out unwanted asset classes
        df = df[(df['asset_class_x'] != 'Loan') & (df['asset_class_y'] != 'Loan')]

        # Calculate market value
        df['market_value'] = df['quantity'] * df['mtm_price'] * df.get('base_factor', 1)
        logging.info("Calculated market value.")

        # Group by report_date and filter_column, then sum the market values
        grouped_data = (
            df.groupby(['report_date', filter_column])['market_value']
            .sum()
            .reset_index()
        )
        grouped_data.rename(columns={'market_value': 'total_market_value'}, inplace=True)

        # Resample data to monthly frequency for each group
        grouped_data['report_date'] = pd.to_datetime(grouped_data['report_date'])
        grouped_data.set_index('report_date', inplace=True)

        monthly_performance = grouped_data.groupby(filter_column).resample('M').agg({
            'total_market_value': ['first', 'last']
        })

        # Flatten MultiIndex columns
        monthly_performance.columns = ['start_value', 'end_value']

        # Calculate monthly returns
        monthly_performance['monthly_returns'] = (
            (monthly_performance['end_value'] - monthly_performance['start_value']) / monthly_performance['start_value'] * 100
        )

        # Reset index for final output
        monthly_performance.reset_index(inplace=True)
        monthly_performance.dropna(subset=['monthly_returns'], inplace=True)

        # Add formatted month column
        monthly_performance['month'] = monthly_performance['report_date'].dt.strftime('%b')

        # Prepare final output
        results = monthly_performance[[
            filter_column, 'month', 'start_value', 'end_value', 'monthly_returns'
        ]].rename(columns={
            'monthly_returns': 'return_value'
        })

        # Convert to JSON format
        json_output = results.to_json(orient='records', double_precision=2)
        logging.info("Converted monthly returns data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        # Return JSON output and execution time
        return json_output, total_time,results

    except Exception as e:
        logging.error("Error processing monthly returns: %s", e)
        raise


**Risk Calculation**
*   Filters out rows where filter_column
*   Find sum of market_value for the selected filter_column
*   call cal_returns() give sthe daily returns
*   Calculate risk by groupby filter column for the daily returns



In [ ]:
def cal_risk(df, filter_column):
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting risk calculation process.")

    try:
        # Ensure 'report_date' is in datetime format
        df['report_date'] = pd.to_datetime(df['report_date'])

        json_output, total_time,daily_returns=cal_returns(df,filter_column)

        logging.info("Calculated daily returns for risk measurement.")

        # Drop NaN values that may arise from pct_change
        daily_returns.dropna(subset=['daily_returns'], inplace=True)

        # Calculate standard deviation of daily returns for each group (risk measure)
        risk_values = daily_returns.groupby(filter_column)['daily_returns'].std().reset_index()
        risk_values.rename(columns={'daily_returns': 'risk'}, inplace=True)

        # Format 'report_date' to match the desired format (e.g., "YYYY-MM-DD")
        daily_returns['report_date'] = daily_returns['report_date'].dt.strftime('%Y-%m-%d')

        # Merge risk values back to daily_returns to have the 'risk' column
        daily_returns = daily_returns.merge(risk_values, on=filter_column, how='left')

        # Convert to JSON format for output
        json_output = daily_returns[['report_date', filter_column, 'total_market_value', 'risk']].to_json(orient='records', double_precision=2)
        logging.info("Converted risk data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        return json_output, total_time,daily_returns

    except Exception as e:
        logging.error("Error processing risk data: %s", e)
        raise

**Sharpe Ratio**
*   Filters out rows where filter_column
*   Find sum of market_value for the selected filter_column
*   call cal_returns() give sthe daily returns
*   Calculate risk by groupby filter column for the daily returns for a month
*   Calculate the sharpe ratio for each month over the specified period


In [ ]:
def cal_sharpe_ratio(df, filter_column, risk_free_rate=0.03):
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting monthly Sharpe Ratio calculation.")

    try:
        # # Ensure 'report_date' is in datetime format and set it as index for resampling
        df['report_date'] = pd.to_datetime(df['report_date'])

        json_output, total_time, daily_returns = cal_returns(df, filter_column)

        # Ensure 'daily_returns' has no NaN values
        daily_returns.dropna(subset=['daily_returns'], inplace=True)

        # Set 'report_date' as the index
        daily_returns.set_index('report_date', inplace=True)

        # Resample by month for each group in filter_column and calculate mean and std dev
        monthly_stats = daily_returns.groupby(filter_column).resample('M')['daily_returns'].agg(['mean', 'std']).reset_index()
        monthly_stats.rename(columns={'mean': 'average_daily_return', 'std': 'risk'}, inplace=True)

        # # Format 'report_date' to match the desired format (e.g., "YYYY-MM-DD")
        monthly_stats['report_date'] = monthly_stats['report_date'].dt.strftime('%Y-%m-%d')

        # Calculate the Sharpe ratio for each group by month
        monthly_stats['sharpe_ratio'] = (monthly_stats['average_daily_return'] - risk_free_rate) / monthly_stats['risk']

        # Convert to JSON format for output
        json_output = monthly_stats[['report_date', filter_column, 'sharpe_ratio']].to_json(orient='records', double_precision=2)
        logging.info("Converted monthly Sharpe Ratio data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        return json_output, total_time, monthly_stats

    except Exception as e:
        logging.error("Error calculating monthly Sharpe Ratio: %s", e)
        raise


**Sortino Ratio**
*   Filters out rows where filter_column
*   Find sum of market_value for the selected filter_column
*   call cal_returns() give sthe daily returns
*   Calculate risk by groupby filter column for the daily returns for a month
*   Calculate the sortino ratio for each month over the specified period

In [ ]:
def cal_sortino_ratio(df, filter_column, target_return=0.0):
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting monthly Sortino Ratio calculation.")

    try:
        # # Ensure 'report_date' is in datetime format
        df['report_date'] = pd.to_datetime(df['report_date'])

        # Call cal_returns function to get daily returns
        json_output, total_time, daily_returns = cal_returns(df, filter_column)

        # Drop NaN values in daily returns
        daily_returns.dropna(subset=['daily_returns'], inplace=True)

        # Set 'report_date' as the index
        daily_returns.set_index('report_date', inplace=True)

        # Filter for only negative returns for downside deviation calculation
        daily_returns['downside_returns'] = daily_returns['daily_returns'].apply(lambda x: min(0, x - target_return))

        # Resample by month for each group in filter_column and calculate average return and downside risk
        monthly_stats = daily_returns.groupby(filter_column).resample('M').agg(
            average_daily_return=('daily_returns', 'mean'),
            downside_risk=('downside_returns', lambda x: (x[x < 0].std()))
        ).reset_index()

        # # Format 'report_date' to "YYYY-MM-DD"
        monthly_stats['report_date'] = monthly_stats['report_date'].dt.strftime('%Y-%m-%d')

        # Calculate the Sortino ratio for each group by month
        monthly_stats['sortino_ratio'] = (monthly_stats['average_daily_return'] - target_return) / monthly_stats['downside_risk']

        # Convert to JSON format for output
        json_output = monthly_stats[['report_date', filter_column, 'sortino_ratio']].to_json(orient='records', double_precision=2)
        logging.info("Converted monthly Sortino Ratio data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        return json_output, total_time, monthly_stats

    except Exception as e:
        logging.error("Error calculating monthly Sortino Ratio: %s", e)
        raise


**Networth**

In [ ]:
def cal_networth(start_date,end_date,df, filter_column):
    """Calculate net worth grouped by asset class and report date."""
    start_time = time.time()
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting net worth calculation process.")

    try:
        # Load the data
        # df = get_data(filepath1)
        # if df is None:
        #     raise ValueError("Data loading failed due to missing columns.")

        # Ensure 'report_date' is in datetime format
        df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')

        # Generate month-end dates automatically for the desired range
        month_end_dates = pd.date_range(start_date,end_date , freq='M')

        # Filter for rows with dates matching the generated month-end dates
        filtered_df = df[df['report_date'].isin(month_end_dates)]

        # Group by asset_class and report_date, then sum market_value
        result_df = (
            filtered_df.groupby(['report_date',filter_column])['market_value']
            .sum()
            .reset_index()
        )

        # Rename columns for clarity
        result_df.columns = ['month_end_date', filter_column, 'Networth']

        logging.info("Grouped data by month-end date and filter column.")

        # Format 'month_end_date' to match the desired format (e.g., "YYYY-MM-DD")
        result_df['month_end_date'] = result_df['month_end_date'].dt.strftime('%Y-%m-%d')

        # Convert to JSON format for output
        json_output = result_df.to_json(orient='records', double_precision=2)
        logging.info("Converted net worth data to JSON format.")

        end_time = time.time()
        total_time = end_time - start_time
        logging.info(f"Total execution time: {total_time:.2f} seconds")

        return json_output, total_time, result_df

    except Exception as e:
        logging.error("Error processing net worth data: %s", e)
        raise


**Main function with filter**
*   This will call all the core functions
*   And we will take input from user to apply the filters only for Assset class Equity and Bonds over a period
*  We will print the json for each core function along with the execution time

In [84]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import logging
import time
import warnings
import quantstats as qs  # For enhanced financial statistics

warnings.filterwarnings("ignore")

# Initialize logging once
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main_stat():
    """Run core functions with sample data."""
    filepath1 = "/content/KP_CM_POSITION_HISTORY.csv"
    filepath2 = "/content/SECURITY_SHEET.csv"
    filepath3="/content/Mutual Funds dataset.csv"

    try:
        options = input("Mutual Funds or Others...")
        if options=="Mutual Funds":
           merged_data=get_data(filepath3)
                      # Convert 'report_date' to datetime format (ensure consistent format)
           merged_data['report_date'] = pd.to_datetime(merged_data['report_date'], errors='coerce')

           # Define filter dictionary to include start_date, end_date, and asset_class_x
           filters = {
            "start_date": '2023-11-01',
            "end_date": '2024-12-31',
            "asset_class_x": ["Mutual Funds & Equivalent"],
            #"client_id":['dda49fb1-4172-4dc3-bbe0-97e9e6cbb873','a5a447c6-318f-480a-8bc0-cc037c0b31db']
            }

        else:
            # Load and merge data
            merged_data = merge_pos_security_sheet(filepath1, filepath2)

           # Convert 'report_date' to datetime format (ensure consistent format)
            merged_data['report_date'] = pd.to_datetime(merged_data['report_date'], errors='coerce')

           # Define filter dictionary to include start_date, end_date, and asset_class_x
            filters = {
            "start_date": '2023-01-01',
            "end_date": '2024-06-30',
            "asset_class_x": ["Equity & Equivalents", "Bond & Equivalents"],
            #"client_id":['dda49fb1-4172-4dc3-bbe0-97e9e6cbb873','a5a447c6-318f-480a-8bc0-cc037c0b31db']
            }

        # Convert start_date and end_date to datetime
        start_date = pd.to_datetime(filters["start_date"])
        end_date = pd.to_datetime(filters["end_date"])

        # Filter the data by report_date range
        merged_data = merged_data[(merged_data['report_date'] >= start_date) & (merged_data['report_date'] <= end_date)]
        logging.info(f"Data filtered between {start_date.date()} and {end_date.date()}.")

        # Initialize filter_column to ensure it is defined
        filter_column = None

        # Get filter column input
        choice = int(input("Enter your choice -\n1. Asset Class\n2. Sub Asset Class\n3. Industry\n4. Region\n"))

        # Map user choice to the corresponding filter column
        if choice == 1:
            filter_column = 'asset_class_x'
        elif choice == 2:
            filter_column = 'sub_asset_class'
        elif choice == 3:
            filter_column = 'sub_industry'
        elif choice == 4:
            filter_column = 'country_name'
        else:
            logging.error("Invalid choice entered. Exiting.")
            return  # Exit early if the choice is invalid

        # Apply the selected filter if filter_column is valid
        if filter_column in filters:
            if filter_column == "asset_class_x":
                merged_data = merged_data[merged_data[filter_column].isin(filters[filter_column])]
                logging.info(f"Data filtered based on {filter_column}: {filters[filter_column]}")

        networth_output, execution_time, networth_stats = cal_networth(start_date,end_date,merged_data, filter_column)        # Print JSON output and execution time
        print("Execution Time:", execution_time, "seconds")
        print("Networth:", networth_output)
        print("\n")

        # Calculate and display returns
        returns_json, returns_time, daily_returns = cal_returns(merged_data, filter_column)
        print("Execution Time:", returns_time, "seconds")
        print("Daily Returns:", returns_json)
        top_62_rows=daily_returns.head(62)
        print("\n")
        # top_62_rows.to_csv('top_62_data.csv', index=False)
        # print("Top 62 rows saved to 'top_62_data.csv'.")


        # Calculate and display returns
        returns_json, returns_time, results = cal_monthly_returns(merged_data, filter_column)
        print("Execution Time:", returns_time, "seconds")
        print("Monthly Returns:", returns_json)
        print("\n")

        # Calculate and display risk
        risk_json, risk_time, daily_returns = cal_risk(merged_data, filter_column)
        print("Execution Time:", risk_time, "seconds")
        print("Risk:", risk_json)
        print("\n")

        # Calculate and display Sharpe Ratio
        sharpe_json, sharpe_time, monthly_stats = cal_sharpe_ratio(merged_data, filter_column)
        print("Execution Time:", sharpe_time, "seconds")
        print("Sharpe Ratio:", sharpe_json)
        print("\n")

        # Calculate and display Sortino Ratio
        sortino_json, sortino_time, monthly_stats = cal_sortino_ratio(merged_data, filter_column)
        print("Execution Time:", sortino_time, "seconds")
        print("Sortino Ratio:", sortino_json)
        print("\n")

        # # Calculate and display Sortino Ratio
        # cagr = cal_cagr(merged_data, filter_column)
        # print("Execution Time:", sortino_time, "seconds")
        # print("CAGR:", cagr)
        # print("\n")

    except Exception as e:
        logging.error("An error occurred in main_stat function: %s", e)

# Function call
main_stat()


Mutual Funds or Others...Mutual Funds
Enter your choice -
1. Asset Class
2. Sub Asset Class
3. Industry
4. Region
1
Execution Time: 0.0414891242980957 seconds
Networth: [{"month_end_date":"2023-11-30","asset_class_x":"Mutual Funds & Equivalent","Networth":126908916.7},{"month_end_date":"2023-12-31","asset_class_x":"Mutual Funds & Equivalent","Networth":65172546.96},{"month_end_date":"2024-01-31","asset_class_x":"Mutual Funds & Equivalent","Networth":128607780.49},{"month_end_date":"2024-02-29","asset_class_x":"Mutual Funds & Equivalent","Networth":129467278.51},{"month_end_date":"2024-03-31","asset_class_x":"Mutual Funds & Equivalent","Networth":129994375.63},{"month_end_date":"2024-04-30","asset_class_x":"Mutual Funds & Equivalent","Networth":130896057.23},{"month_end_date":"2024-05-31","asset_class_x":"Mutual Funds & Equivalent","Networth":131636070.96},{"month_end_date":"2024-06-30","asset_class_x":"Mutual Funds & Equivalent","Networth":100759691.37},{"month_end_date":"2024-07-31","

In [ ]:
import pandas as pd
import numpy as np
import quantstats as qs
import logging

# Function to calculate CAGR
def cal_cagr(df, filter_column):
    try:
        logging.info("Calculating CAGR.")

        # Calculate daily returns using cal_returns
        daily_returns = cal_returns(df, filter_column)

        # Ensure 'daily_returns' column exists and drop NaN
        daily_returns.dropna(subset=['daily_returns'], inplace=True)

        # Group by the filter column to calculate CAGR for each group
        cagr_values = {}
        for group in daily_returns[filter_column].unique():
            group_returns = daily_returns[daily_returns[filter_column] == group]['daily_returns']
            cagr_values[group] = qs.stats.cagr(group_returns, rf=0, compounded=True, periods=252)  # Assuming 252 trading days

        logging.info("CAGR calculated successfully.")
        return cagr_values

    except Exception as e:
        logging.error("Error calculating CAGR: %s", e)
        raise

# Function to calculate rolling returns
def cal_rolling_returns(df, filter_column):
    try:
        logging.info("Calculating rolling returns.")

        # Calculate daily returns using cal_returns
        daily_returns = cal_returns(df, filter_column)

        # Ensure 'daily_returns' column exists and drop NaN
        daily_returns.dropna(subset=['daily_returns'], inplace=True)

        # Calculate rolling returns for a 3-month period (63 trading days)
        rolling_returns = daily_returns.groupby(filter_column)['daily_returns'].rolling(window=63).mean().reset_index()
        rolling_returns.rename(columns={'daily_returns': 'rolling_returns'}, inplace=True)

        logging.info("Rolling returns calculated successfully.")
        return rolling_returns

    except Exception as e:
        logging.error("Error calculating rolling returns: %s", e)
        raise

# Function to calculate beta
def cal_beta(df, benchmark_df, filter_column):
    try:
        logging.info("Calculating beta.")

        # Calculate returns for portfolio and benchmark
        portfolio_returns = cal_returns(df, filter_column)
        benchmark_returns = cal_returns(benchmark_df, filter_column)

        # Align the data by date
        merged_returns = pd.merge(portfolio_returns, benchmark_returns, on='report_date', suffixes=('_portfolio', '_benchmark'))

        # Group by the filter column to calculate beta for each group
        beta_values = {}
        for group in merged_returns[filter_column].unique():
            group_data = merged_returns[merged_returns[filter_column] == group]
            beta = qs.stats.greeks(group_data['daily_returns_portfolio'], group_data['daily_returns_benchmark']).to_dict().get('beta', 0)
            beta_values[group] = beta

        logging.info("Beta calculated successfully.")
        return beta_values

    except Exception as e:
        logging.error("Error calculating beta: %s", e)
        raise

# Function to calculate alpha
def cal_alpha(df, benchmark_df, filter_column):
    try:
        logging.info("Calculating alpha.")

        # Calculate returns for portfolio and benchmark
        portfolio_returns = cal_returns(df, filter_column)
        benchmark_returns = cal_returns(benchmark_df, filter_column)

        # Align the data by date
        merged_returns = pd.merge(portfolio_returns, benchmark_returns, on='report_date', suffixes=('_portfolio', '_benchmark'))

        # Group by the filter column to calculate alpha for each group
        alpha_values = {}
        for group in merged_returns[filter_column].unique():
            group_data = merged_returns[merged_returns[filter_column] == group]
            beta = qs.stats.greeks(group_data['daily_returns_portfolio'], group_data['daily_returns_benchmark']).to_dict().get('beta', 0)
            alpha = group_data['daily_returns_portfolio'].mean() - beta * group_data['daily_returns_benchmark'].mean()
            alpha = alpha * 252  # Annualizing alpha assuming 252 trading days
            alpha_values[group] = alpha

        logging.info("Alpha calculated successfully.")
        return alpha_values

    except Exception as e:
        logging.error("Error calculating alpha: %s", e)
        raise


In [ ]:
# Ensure 'report_date' is the index and sorted

df.set_index('report_date', inplace=True)
df.index = pd.to_datetime(df.index)

# Calculate Daily Returns
df['daily_returns'] = qs.utils.to_returns(df['networth'])

# MTD Calculation
df['MTD'] = df.groupby([df.index.year, df.index.month])['daily_returns'].cumsum()

# QTD Calculation
df['QTD'] = df.groupby([df.index.year, df.index.quarter])['daily_returns'].cumsum()

# YTD Calculation
df['YTD'] = df.groupby(df.index.year)['daily_returns'].cumsum()

# Convert Back to Cumulative Performance (Optional)
df['MTD_Performance'] = qs.utils.cum_returns(df['MTD'])
df['QTD_Performance'] = qs.utils.cum_returns(df['QTD'])
df['YTD_Performance'] = qs.utils.cum_returns(df['YTD'])

print(df[['networth', 'MTD_Performance', 'QTD_Performance', 'YTD_Performance']].head())